In [2]:
import os
import sys
import time
import pickle
import prince
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.cm import get_cmap
from matplotlib.font_manager import FontProperties

# 獲取當前工作目錄
current_dir = os.getcwd()
version3_path = os.path.join(current_dir, "TrafficTDApython", "Version3", "tdamapper", "core_old.py")

from sklearn.utils import resample
from sklearn.cluster import AgglomerativeClustering
from tdamapper.core_old import MapperAlgorithm
from tdamapper.cover import CubicalCover
from tdamapper.clustering import FailSafeClustering
from tdamapper.plot import MapperLayoutInteractive, MapperPlot

from models import *
from utils_v3 import *
from plots import *

try:
    myfont = FontProperties(fname=r"/System/Library/Fonts/PingFang.ttc")
    sns.set(style="whitegrid", font=myfont.get_name())
except Exception as e:
    print(e)

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

[Errno 2] No such file or directory: 'C:\\System\\Library\\Fonts\\PingFang.ttc'


In [3]:
data1 = pd.read_csv("../Data/NPA_TMA2_1.csv", low_memory=False)[:-2]
data2 = pd.read_csv("../Data/NPA_TMA2_2.csv", low_memory=False)[:-2]
data3 = pd.read_csv("../Data/NPA_TMA2_3.csv", low_memory=False)[:-2]
data4 = pd.read_csv("../Data/NPA_TMA2_4_new.csv", low_memory=False)[:-2]
data5 = pd.read_csv("../Data/NPA_TMA2_5.csv", low_memory=False)[:-2]
data6 = pd.read_csv("../Data/NPA_TMA2_6_new.csv", low_memory=False)[:-2]
data7 = pd.read_csv("../Data/NPA_TMA2_7.csv", low_memory=False)[:-2]
data8 = pd.read_csv("../Data/NPA_TMA2_8.csv", low_memory=False)[:-2]
data9 = pd.read_csv("../Data/NPA_TMA2_9.csv", low_memory=False)[:-2]
data10 = pd.read_csv("../Data/NPA_TMA2_10.csv", low_memory=False)[:-2]

dataA2 = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10], ignore_index=True)
# dataA2 = pd.concat([data1, data2, data3, data4, data5], ignore_index=True)

dataA1 = pd.read_csv("../Data/NPA_TMA1_V3.csv")[:-2]

In [6]:
dataA2 = pd.read_csv("../Version3/Data/A2.csv", low_memory=False)
dataA1 = pd.read_csv("../Version3/Data/A1.csv")

In [59]:
# car data
car_0 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_0.csv"), encoding='utf-8')
car_1 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_1.csv"), encoding='utf-8')
car_2 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_2.csv"), encoding='utf-8')
car_3 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_3.csv"), encoding='utf-8')
car_21 = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_21.csv"), encoding='utf-8')
car_out = pd.read_csv(os.path.join(current_dir, "Data/CarData/full_out.csv"), encoding='utf-8')
# pass data
pass_data = pd.read_csv(os.path.join(current_dir, "Data/PassData/full.csv"), encoding='utf-8')
# full data
full_car = pd.concat([car_0, car_1, car_2, car_3, car_21, car_out], axis=0)
full_car['當事者行動狀態大類別名稱'] = 'car'
passforfull = pass_data.copy()
passforfull['當事者行動狀態大類別名稱'] = 'pass'
full_data = pd.concat([full_car, passforfull], axis=0)
car_out = pd.concat([car_out, car_21], axis=0)

In [62]:
select_lst = [
    '天候名稱', '光線名稱', 
    '道路類別-第1當事者-名稱', '速限-第1當事者', 
    '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱',
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',
    '當事者屬-性-別名稱', '當事者事故發生時年齡',
    '保護裝備名稱', '行動電話或電腦或其他相類功能裝置名稱',
    '肇事逃逸類別名稱-是否肇逃',

    # 大類別
    '道路型態大類別名稱', '事故位置大類別名稱',
    '車道劃分設施-分向設施大類別名稱',
    '事故類型及型態大類別名稱', '當事者區分-類別-大類別名稱-車種',
    '車輛撞擊部位大類別名稱-最初', '車輛撞擊部位大類別名稱-其他',

    # 兩個欄位只有兩個觀察值不同
    '肇因研判大類別名稱-主要',
    # '肇因研判大類別名稱-個別',
    '受傷', '死亡'
]

# select data
car_0 = car_0[select_lst]
car_1 = car_1[select_lst]
car_2 = car_2[select_lst]
car_3 = car_3[select_lst]
car_out = car_out[select_lst]
pass_data = pass_data[select_lst]
select_lst.append('當事者行動狀態大類別名稱')
full_data = full_data[select_lst]

In [63]:
# get dummy
car_0 = pd.get_dummies(car_0)
car_1 = pd.get_dummies(car_1)
car_2 = pd.get_dummies(car_2)
car_3 = pd.get_dummies(car_3)
car_out = pd.get_dummies(car_out)
pass_data = pd.get_dummies(pass_data)
full_data = pd.get_dummies(full_data)

In [64]:
print(car_0['死亡'].value_counts())
print(car_1['死亡'].value_counts())
print(car_2['死亡'].value_counts())
print(car_3['死亡'].value_counts())
print(car_out['死亡'].value_counts())
print(pass_data['死亡'].value_counts())
print(full_data['死亡'].value_counts())

0    2910
1      10
Name: 死亡, dtype: int64
0    30072
1      116
2        3
3        1
Name: 死亡, dtype: int64
0    51765
1      214
2        2
Name: 死亡, dtype: int64
0    4905
1      17
Name: 死亡, dtype: int64
0    4336
1      44
2       4
3       2
Name: 死亡, dtype: int64
0    3038
1      57
2       1
Name: 死亡, dtype: int64
0    97026
1      458
2       10
3        3
Name: 死亡, dtype: int64


In [65]:
def get_train_test_data(input_data):
    input_data['y'] = input_data['死亡'].apply(lambda x: 1 if x >= 1 else 0)
    
    new_input_data = input_data.drop(columns=['受傷', '死亡'], inplace=False)
    
    X = new_input_data.drop(columns=['y'])
    y = new_input_data['y']

    return X, y

In [66]:
car_0_X, car_0_y = get_train_test_data(car_0)
car_1_X, car_1_y = get_train_test_data(car_1)
car_2_X, car_2_y = get_train_test_data(car_2)
car_3_X, car_3_y = get_train_test_data(car_3)
car_out_X, car_out_y = get_train_test_data(car_out)
pass_data_X, pass_data_y = get_train_test_data(pass_data)
full_data_X, full_data_y = get_train_test_data(full_data)

In [67]:
car0_logistic_metric, car0_logistic_score, car0_logistic_cm, car0_y_resampled_test, car0_decision_scores = logistic_cm_gridsearch(car_0_X, car_0_y, threshold=5)
car1_logistic_metric, car1_logistic_score, car1_logistic_cm, car1_y_resampled_test, car1_decision_scores = logistic_cm_gridsearch(car_1_X, car_1_y, threshold=5)
car2_logistic_metric, car2_logistic_score, car2_logistic_cm, car2_y_resampled_test, car2_decision_scores = logistic_cm_gridsearch(car_2_X, car_2_y, threshold=5)
car3_logistic_metric, car3_logistic_score, car3_logistic_cm, car3_y_resampled_test, car3_decision_scores = logistic_cm_gridsearch(car_3_X, car_3_y, threshold=5)
carout_logistic_metric, carout_logistic_score, carout_logistic_cm, carout_y_resampled_test, carout_decision_scores = logistic_cm_gridsearch(car_out_X, car_out_y, threshold=5)
pass_logistic_metric, pass_logistic_score, pass_logistic_cm, pass_y_resampled_test, pass_decision_scores = logistic_cm_gridsearch(pass_data_X, pass_data_y, threshold=5)
full_logistic_metric, full_logistic_score, full_logistic_cm, full_y_resampled_test, full_decision_scores = logistic_cm_gridsearch(full_data_X, full_data_y, threshold=5)

Best parameters found by GridSearchCV: {'C': 1, 'penalty': 'l2'}
Best parameters found by GridSearchCV: {'C': 1, 'penalty': 'l2'}
Best parameters found by GridSearchCV: {'C': 1, 'penalty': 'l2'}
Best parameters found by GridSearchCV: {'C': 1, 'penalty': 'l2'}
Best parameters found by GridSearchCV: {'C': 1, 'penalty': 'l2'}
Best parameters found by GridSearchCV: {'C': 1, 'penalty': 'l2'}
Best parameters found by GridSearchCV: {'C': 1, 'penalty': 'l2'}


In [92]:
def get_score(resampled_test, decision_scores, threshold=0.5):
    
    y_pred = (decision_scores >= threshold).astype(int)

    conf_matrix = confusion_matrix(resampled_test, y_pred)
    acc = accuracy_score(resampled_test, y_pred)
    
    return conf_matrix, acc

In [95]:
get_score(car0_y_resampled_test, car0_decision_scores, 0.02)

(array([[2, 2],
        [2, 2]], dtype=int64),
 0.5)

In [83]:
# save car0_y_resampled_test, car0_decision_scores

metrics_df = pd.DataFrame({
    'score': car0_decision_scores,
    'y': car0_y_resampled_test
})
metrics_df.to_csv("./ModelPerformance/car0.csv", index=False)

metrics_df = pd.DataFrame({
    'score': car1_decision_scores,
    'y': car1_y_resampled_test
})
metrics_df.to_csv("./ModelPerformance/car1.csv", index=False)

metrics_df = pd.DataFrame({
    'score': car2_decision_scores,
    'y': car2_y_resampled_test
})
metrics_df.to_csv("./ModelPerformance/car2.csv", index=False)

metrics_df = pd.DataFrame({
    'score': car3_decision_scores,
    'y': car3_y_resampled_test
})
metrics_df.to_csv("./ModelPerformance/car3.csv", index=False)

metrics_df = pd.DataFrame({
    'score': carout_decision_scores,
    'y': carout_y_resampled_test
})
metrics_df.to_csv("./ModelPerformance/carout.csv", index=False)

metrics_df = pd.DataFrame({
    'score': pass_decision_scores,
    'y': pass_y_resampled_test
})
metrics_df.to_csv("./ModelPerformance/pass.csv", index=False)

metrics_df = pd.DataFrame({
    'score': full_decision_scores,
    'y': full_y_resampled_test
})
metrics_df.to_csv("./ModelPerformance/full.csv", index=False)

In [88]:
car0_decision_scores

array([0.00739187, 0.01219601, 0.0215474 , 0.02724232, 0.04964881,
       0.00894119, 0.02023626, 0.01526169])